---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    import pandas as pd 
    import numpy as np
    lines = []
    with open('university_towns.txt', 'r') as file_to_read:
        while True:
            line = file_to_read.readline()
            if not line:
                break
            line = line.strip('\n')
            lines.append(line)
    state_name=[]
    regions = []
    for line in lines:
        if line[-6:] != '[edit]':
            line = line.split('(')[0].strip(' ')
            tmp.append(line)        
        else: 
            tmp = []
            regions.append(tmp)
            line = line[:-6]
            state_name.append(line)      
    result = dict(zip(state_name, regions))
    df = pd.DataFrame(state_name, columns = ['states'])
    df['regions'] = df['states'].map(lambda x: result[x])
    df=pd.DataFrame({'states':df.states.repeat(df.regions.str.len()),'regions':np.concatenate(df.regions.values)})
    df = df.rename(columns={'states':'State','regions':'RegionName',})
#     new_state = {v : k for k, v in states.items()}
#     df['State'] = df['State'].map(lambda x: new_state[x])
    a = df.pop('RegionName')
    df['RegionName'] = a 
    return df

In [ ]:
def get_recession_start():
    GDP = pd.read_excel('gdplev.xls', skiprows = 4)
    GDP.head()
    GDP = GDP.iloc[:,4:]
    GDP.columns = ['quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars', 'NA']
    GDP_SUB = GDP.iloc[3:,:]
    df = GDP_SUB.drop(['GDP in billions of current dollars','NA'], axis = 1)
    df = df.iloc[212:,:]
    is_up = df.iloc[1,1] > df.iloc[0,1]
    prev_value = df.iloc[0,1]
    cnt = 1 
    result = []
    for i in range(2,len(df)):
        if is_up == (df.iloc[i,1] > prev_value):
            cnt += 1 
        else: 
            result.append([is_up, cnt, df.iloc[i-1,0]])
            is_up = not is_up
            cnt = 1 
        prev_value = df.iloc[i, 1]
    result.append([is_up, cnt, df.iloc[-1,0]])
    start = ''
    for i in range(len(result)):
        if result[i][0] == False and result[i][1] >= 2 and result[i+1][0] == True and result[i+1][1] >= 2:
            start = result[i-1][2]
    a = start.split('q', 1)
    quarter = [int(a[1])+1 if int(a[1]) < 4 else 1]
    year = [int(a[0]) if int(a[1]) < 4 else int(a[0]) +1]
    q_y = str(year[0]) + 'q' + str(quarter[0])
    return q_y

In [ ]:
def get_recession_end():
    GDP = pd.read_excel('gdplev.xls', skiprows = 4)
    GDP.head()
    GDP = GDP.iloc[:,4:]
    GDP.columns = ['quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars', 'NA']
    GDP_SUB = GDP.iloc[3:,:]
    df = GDP_SUB.drop(['GDP in billions of current dollars','NA'], axis = 1)
    df = df.iloc[212:,:]
    is_up = df.iloc[1,1] > df.iloc[0,1]
    prev_value = df.iloc[0,1]
    cnt = 1 
    result = []
    for i in range(2, len(df)):
        if is_up == (df.iloc[i,1] > prev_value):
            cnt += 1 
        else: 
            result.append([is_up, cnt, df.iloc[i-1,0]])
            is_up = not is_up
            cnt = 1 
        prev_value = df.iloc[i, 1]
    result.append([is_up, cnt, df.iloc[-1,0]])
    end = ''
    for i in range(len(result)):
        if result[i][0] == False and result[i][1] >= 2 and result[i+1][0] == True and result[i+1][1] >= 2:
            end = result[i][2]
    a = end.split('q', 1)
    quarter = [int(a[1])+2 if int(a[1]) <= 2 else 1]
    year = [int(a[0]) if int(a[1]) <=2 else int(a[0]) +1]
    q_y = str(year[0]) + 'q' + str(quarter[0])    
    return q_y

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel('gdplev.xls', skiprows = 4)
    GDP.head()
    GDP = GDP.iloc[:,4:]
    GDP.columns = ['quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars', 'NA']
    GDP_SUB = GDP.iloc[3:,:]
    df = GDP_SUB.drop(['GDP in billions of current dollars','NA'], axis = 1)
    df = df.iloc[212:,:]
    is_up = df.iloc[1,1] > df.iloc[0,1]
    prev_value = df.iloc[0,1]
    cnt = 1 
    result = []
    for i in range(2, len(df)):
        if is_up == (df.iloc[i,1] > prev_value):
            cnt += 1 
        else: 
            result.append([is_up, cnt, df.iloc[i-1,0]])
            is_up = not is_up
            cnt = 1 
        prev_value = df.iloc[i, 1]
    result.append([is_up, cnt, df.iloc[-1,0]])
    bottom = ''
    for i in range(len(result)):
        if result[i][0] == False and result[i][1] >= 2 and result[i+1][0] == True and result[i+1][1] >= 2:
            bottom = result[i][2] 
    return bottom

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    house = df[['RegionName', 'State'] + list(df.columns[51:])]
    house['State'] = house['State'].map(lambda x:states[x])
    house = house.set_index(['State','RegionName'])
    house['2016-09'] = house.iloc[:, -2:].mean(axis=1)
    m, n = house.shape
    house_np = house.values.reshape(m, n//3, 3)
    house_np_mean = np.mean(house_np, axis=-1)
    house_mean = pd.DataFrame(house_np_mean)
    q_names = ['{}q{}'.format(y, q) for y in range(2000, 2016) for q in range(1, 5)] + ['2016q1', '2016q2', '2016q3']
    house_mean.columns = q_names
    house_mean.index = house.index
    return house_mean

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    import pandas as pd
    from scipy import stats
    df = convert_housing_data_to_quarters()
    df_copy = df.iloc[:, 33:40]
    df_copy['start_bott_diff'] = df_copy['2009q2'] - df_copy['2008q3']
    df_copy['price_ratio'] = df_copy['2008q2']/df_copy['2009q2']
    univ_town = get_list_of_university_towns()
    univ_town['label'] = '1'
    df_copy = df_copy.reset_index()
    join_data = pd.merge(df_copy, univ_town, how = 'left', left_on = ['State', 'RegionName'], right_on = ['State', 'RegionName'])
    univ_mean = join_data[join_data['label'] == '1']['price_ratio']
    non_univ_mean = join_data[join_data['label'] != '1']['price_ratio']
    k = stats.ttest_ind(univ_mean, non_univ_mean, nan_policy = "omit") 
    tmp = ('university town' if univ_mean.mean() < non_univ_mean.mean() else 'non-university town')
    result = (k.pvalue<0.01, k.pvalue, tmp)
    return result